In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [47]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, conv2_in, h_dim3, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        #new part of encoder
        self.fcn3 = nn.Linear(h_dim2, conv2_in)
        self.fcn5 = nn.Linear(conv2_in, h_dim3)
        #end new part of encoder
        self.fc31 = nn.Linear(h_dim3, z_dim)
        self.fc32 = nn.Linear(h_dim3, z_dim)


        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim3)
        #start new part of decoder
        self.fcn6 = nn.Linear(h_dim3, conv2_in)
        self.fcn7 = nn.Linear(conv2_in, h_dim2)
        #end new part of decoder
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        h = F.relu(self.fcn3(h))
        h = F.relu(self.fcn5(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fcn6(h))
        h = F.relu(self.fcn7(h))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, conv2_in=64, h_dim3 = 16,z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [48]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fcn3): Linear(in_features=256, out_features=64, bias=True)
  (fcn5): Linear(in_features=64, out_features=16, bias=True)
  (fc31): Linear(in_features=16, out_features=2, bias=True)
  (fc32): Linear(in_features=16, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=16, bias=True)
  (fcn6): Linear(in_features=16, out_features=64, bias=True)
  (fcn7): Linear(in_features=64, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [49]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [50]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        #data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [51]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            #data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [52]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.709336
Train Epoch: 1 [10000/60000 (17%)]	Loss: 202.059844
Train Epoch: 1 [20000/60000 (33%)]	Loss: 197.058262
Train Epoch: 1 [30000/60000 (50%)]	Loss: 191.318203
Train Epoch: 1 [40000/60000 (67%)]	Loss: 181.050723
Train Epoch: 1 [50000/60000 (83%)]	Loss: 167.703711
====> Epoch: 1 Average loss: 190.9503
====> Test set loss: 166.4835
Train Epoch: 2 [0/60000 (0%)]	Loss: 161.607812
Train Epoch: 2 [10000/60000 (17%)]	Loss: 161.872754
Train Epoch: 2 [20000/60000 (33%)]	Loss: 165.943809
Train Epoch: 2 [30000/60000 (50%)]	Loss: 152.584180
Train Epoch: 2 [40000/60000 (67%)]	Loss: 160.604170
Train Epoch: 2 [50000/60000 (83%)]	Loss: 163.981152
====> Epoch: 2 Average loss: 161.2531
====> Test set loss: 156.7104
Train Epoch: 3 [0/60000 (0%)]	Loss: 156.870869
Train Epoch: 3 [10000/60000 (17%)]	Loss: 154.982637
Train Epoch: 3 [20000/60000 (33%)]	Loss: 159.904561
Train Epoch: 3 [30000/60000 (50%)]	Loss: 149.999063
Train Epoch: 3 [40000/60000 (67%)]	Loss: 153.20

In [53]:
with torch.no_grad():
    z = torch.randn(64, 2)
    sample = vae.decoder(z)
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')